# Rock, Scissor, Paper Classifier

In [1]:
#################
### Libraries ###
#################
from PIL import Image
import os, glob, shutil, random
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras

## 1차 테스트 결과

################################################################################################
- 먼저, 노드에서 데이터셋을 수집 후 분류기 만들고 간단하게 테스트해 본 결과, 테스트 정확도: **41%**
- 정확도가 낮은 원인으로 아래 두 가지라고 판단
    1. 학습을 위한 데이터가 적음 (training dataset으로 각 100장)
    2. 먼저 수집한 데이터를 학습용으로 사용하고, 이후에 수집한 데이터를 테스트용으로 사용한다면 문제가 될 수 있음    
(ref. CS231n: Lecture2 | Image Classification)

################################################################################################

## 2차 테스트 진행

목표
- 데이터 추가(Data Augmentation) 및 전체 데이터셋을 무작위로 섞고 분할(Data Splitting)하여 테스트한 결과 비교    

In [2]:
###########################################################
### Data Splitting: From full_set to train_set/test_set ###
###########################################################
"""
( 가지고 있는 데이터셋에 맞게 아래 1~2번 수작업 필요 )
1. 가지고 있는 rock, scissor, paper 모든 데이터를 \full_set(폴더 만든 후)으로 이동
2. 모든 파일명을 'blabla (###).jpg' 형태로 바꿔줄 것: 'Ctrl+A'(모든파일 선택)->'F2'(이름 바꾸기)
"""
full_dir = r"C:\Users\Jaewoong\Desktop\AIFFEL\Exploration\rock_scissor_paper\full_set"
base_dir = os.path.dirname(full_dir)
train_dir = base_dir+r"\train_set"
test_dir = base_dir+r"\test_set"
if os.path.isdir(train_dir):
    shutil.rmtree(train_dir)
if os.path.isdir(test_dir):
    shutil.rmtree(test_dir)

# train_set으로 full_set을 전체 복사
shutil.copytree(full_dir, train_dir)

n_test_files_per_label = 100
label_lst = ['rock', 'scissor', 'paper']

for label in label_lst:
	if not os.path.isdir(test_dir+f"\{label}"):
		os.makedirs(test_dir+f"\{label}")
	full_files = glob.glob(train_dir+f"\{label}\*.jpg")
	n_train_files_per_label = len(full_files)	- n_test_files_per_label

	# 전체 데이터셋(full_set)에서 test_set을 random sampling
	test_files = random.sample(full_files, n_test_files_per_label)
	if not set(test_files) == {0}:
		for file_to_move in test_files:
			shutil.move(file_to_move, test_dir+f"\{label}")
	
total_num_train_set = n_train_files_per_label*len(label_lst)
total_num_test_set = n_test_files_per_label*len(label_lst)

In [3]:
#####################
### Resize images ###
#####################
def resize_images(img_path):
	images=glob.glob(img_path + "/*.jpg")  
    
	print(len(images), " images to be resized.")

    # 파일마다 모두 28x28 사이즈로 바꾸어 저장
	target_size=(28,28)
	for img in images:
		old_img=Image.open(img)
		new_img=old_img.resize(target_size,Image.ANTIALIAS)
		new_img.save(img, "JPEG")
    
	print(len(images), " images resized.")
	
# 각 dataset 이미지가 저장된 디렉토리 아래의 모든 jpg 파일 resizing
for label in label_lst:
	train_img_path = train_dir+f"\{label}"
	test_img_path = test_dir+f"\{label}"
	resize_images(train_img_path)
	print(f"The size of {label.upper()} train_set is 28x28!")

	resize_images(test_img_path)
	print(f"The size of {label.upper()} test_set is 28x28!")

1000  images to be resized.
1000  images resized.
The size of ROCK train_set is 28x28!
100  images to be resized.
100  images resized.
The size of ROCK test_set is 28x28!
1000  images to be resized.
1000  images resized.
The size of SCISSOR train_set is 28x28!
100  images to be resized.
100  images resized.
The size of SCISSOR test_set is 28x28!
1000  images to be resized.
1000  images resized.
The size of PAPER train_set is 28x28!
100  images to be resized.
100  images resized.
The size of PAPER test_set is 28x28!


In [4]:
##########################
### Load train dataset ###
##########################
def load_data(img_path, number_of_data=6000):  # 가위바위보 이미지 개수 총합에 주의하세요.
    # 가위 : 0, 바위 : 1, 보 : 2
    img_size=28
    color=3
    #이미지 데이터와 라벨(가위 : 0, 바위 : 1, 보 : 2) 데이터를 담을 행렬(matrix) 영역을 생성합니다.
    imgs=np.zeros(number_of_data*img_size*img_size*color,dtype=np.int32).reshape(number_of_data,img_size,img_size,color)
    labels=np.zeros(number_of_data,dtype=np.int32)

    idx=0
    for file in glob.iglob(img_path+'/scissor/*.jpg'):
        img = np.array(Image.open(file),dtype=np.int32)
        imgs[idx,:,:,:]=img    # 데이터 영역에 이미지 행렬을 복사
        labels[idx]=0   # 가위 : 0
        idx=idx+1

    for file in glob.iglob(img_path+'/rock/*.jpg'):
        img = np.array(Image.open(file),dtype=np.int32)
        imgs[idx,:,:,:]=img    # 데이터 영역에 이미지 행렬을 복사
        labels[idx]=1   # 바위 : 1
        idx=idx+1  
    
    for file in glob.iglob(img_path+'/paper/*.jpg'):
        img = np.array(Image.open(file),dtype=np.int32)
        imgs[idx,:,:,:]=img    # 데이터 영역에 이미지 행렬을 복사
        labels[idx]=2   # 보 : 2
        idx=idx+1
    return imgs, labels

# load image data
(x_train, y_train)=load_data(train_dir, number_of_data=total_num_train_set)

# normalization
x_train_norm = x_train/255.0   # 입력은 0~1 사이의 값으로 정규화

print("x_train shape: {}".format(x_train.shape))
print("y_train shape: {}".format(y_train.shape))

x_train shape: (3000, 28, 28, 3)
y_train shape: (3000,)


In [5]:
############################
### Design train network ###
############################
n_channel_1=32
n_channel_2=32
n_dense=64
n_train_epoch=10

model=keras.models.Sequential()
model.add(keras.layers.Conv2D(n_channel_1, (3,3), activation='relu', input_shape=(28,28,3)))
model.add(keras.layers.MaxPool2D(2,2))
model.add(keras.layers.Conv2D(n_channel_2, (3,3), activation='relu'))
model.add(keras.layers.MaxPooling2D((2,2)))
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(n_dense, activation='relu'))
model.add(keras.layers.Dense(3, activation='softmax'))

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 32)        896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 11, 11, 32)        9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 5, 5, 32)          0         
_________________________________________________________________
flatten (Flatten)            (None, 800)               0         
_________________________________________________________________
dense (Dense)                (None, 64)                51264     
_________________________________________________________________
dense_1 (Dense)              (None, 3)                 1

In [6]:
###################
### Train model ###
###################
model.compile(optimizer='adam',
            loss='sparse_categorical_crossentropy',
            metrics=['accuracy'])
model.fit(x_train_norm, y_train, epochs=n_train_epoch)

Epoch 1/10
94/94 [==============================] - 0s 3ms/step - loss: 1.0744 - accuracy: 0.3947
Epoch 2/10
94/94 [==============================] - 0s 3ms/step - loss: 0.9158 - accuracy: 0.5643
Epoch 3/10
94/94 [==============================] - 0s 2ms/step - loss: 0.6258 - accuracy: 0.7467
Epoch 4/10
94/94 [==============================] - 0s 2ms/step - loss: 0.4196 - accuracy: 0.8473
Epoch 5/10
94/94 [==============================] - 0s 2ms/step - loss: 0.2941 - accuracy: 0.9050
Epoch 6/10
94/94 [==============================] - 0s 2ms/step - loss: 0.2228 - accuracy: 0.9387
Epoch 7/10
94/94 [==============================] - 0s 2ms/step - loss: 0.1468 - accuracy: 0.9653
Epoch 8/10
94/94 [==============================] - 0s 2ms/step - loss: 0.1206 - accuracy: 0.9687
Epoch 9/10
94/94 [==============================] - 0s 2ms/step - loss: 0.0909 - accuracy: 0.9777
Epoch 10/10
94/94 [==============================] - 0s 3ms/step - loss: 0.0802 - accuracy: 0.9787


In [7]:
#########################
### Load test dataset ###
#########################
# test case 1 (new dataset)
new_test_dir = r"C:\Users\Jaewoong\Desktop\AIFFEL\Exploration\data\rock_scissor_paper\rock_scissor_paper_test"
(x_test, y_test)=load_data(new_test_dir, number_of_data=300)
x_test_norm = x_test/255.0

# test case 2 (Split dataset)
(x_test_split, y_test_split)=load_data(test_dir, number_of_data=total_num_test_set)
x_test_split_norm = x_test_split/255.0
print('\n')

##########################
### Test trained model ###
##########################
# test case 1 (new dataset)
print('-------------------- test case 1 (New dataset) --------------------')
test_loss, test_accuracy = model.evaluate(x_test_norm, y_test, verbose=2)
print("Hyperparams... n_ch_1: {}, n_ch_2: {}, n_dense: {}, n_train_epoch: {}".format(n_channel_1, n_channel_2, n_dense, n_train_epoch))
print("test_loss: {} ".format(test_loss))
print("test_accuracy: {}".format(test_accuracy))
print('\n')

# test case 2 (Split dataset)
print('-------------------- test case 2 (Split dataset) --------------------')
test_loss, test_accuracy = model.evaluate(x_test_split_norm, y_test_split, verbose=2)
print("Hyperparams... n_ch_1: {}, n_ch_2: {}, n_dense: {}, n_train_epoch: {}".format(n_channel_1, n_channel_2, n_dense, n_train_epoch))
print("test_loss: {} ".format(test_loss))
print("test_accuracy: {}".format(test_accuracy))



-------------------- test case 1 (New dataset) --------------------
10/10 - 0s - loss: 1.1415 - accuracy: 0.6400
Hyperparams... n_ch_1: 32, n_ch_2: 32, n_dense: 64, n_train_epoch: 10
test_loss: 1.1415170431137085 
test_accuracy: 0.6399999856948853


-------------------- test case 2 (Split dataset) --------------------
10/10 - 0s - loss: 0.1121 - accuracy: 0.9600
Hyperparams... n_ch_1: 32, n_ch_2: 32, n_dense: 64, n_train_epoch: 10
test_loss: 0.11205104738473892 
test_accuracy: 0.9599999785423279


## 2차 테스트 결과   

################################################################################################
- 다양한 데이터 추가(1000장)로 모델 성능 향상: 전혀 상관없는 test_set(case#1)으로는 **64%**, 분할한 test_set(case#2)으로는 **95%**
- 기존의 적은 학습 데이터셋(100장)으로는 모델이 어느 특정 부분에만 과적합(overfitting)되었다고 판단
- 또 다른 한 가지, test case 2 (Split dataset) 결과가 정말 정확한 것인지? 정확도가 너무 높게 나와서 의심...
    - 연속 촬영 이미지이므로 random sample 이라도 사실상 비슷한 이미지일 수 있음

################################################################################################

## 3차 테스트 진행

목표
- Overfitting 개선 방법인 **Regularization**과 **Dropout**을 적용     
- hyperparameter tuning 을 위해 train & evaluate 통합 (classification_model 함수)

**How to Prevent Overfitting**
1. Simplify Model
    - 모델의 complexcity를 줄이자. network를 구성하는 layer 수 or 뉴런 수(특징 수) 줄여보기
2. Early Stopping
    - 일정 training iteration이 지나면 training error는 계속 줄어들지라도 test error가 늘어나기 시작함
    - test error가 늘어나기 전에 학습을 중지시키기 위해 train_epoch를 크게 늘리지 않기
3. Use Data Augmentation
    - flipped, translated, rotated, scaled train_set을 늘려서 overffiting 방지
4. Use Regularization
    - L1/L2 regularizer을 활용하여 weight에 제한(규제)을 두므로 모델의 일반화 효과를 가져옴
    - Keras를 사용한다면, Layer weight regularizers (3 keyword arguments)
        - kernel_regularizer : 가중치 규제 (사용)
        - bias_regularizer : 편향 규제
        - activity_regularizer : 출력 값 규제
5. Use Dropouts
    - 학습 시 신경망이 특정 뉴런 또는 특정 조합에 너무 의존적이게 되는 것을 방지 (랜덤성/다양성 증가)
    - Regularization 효과up (L1/L2는 cost function(small noise)을 수정 하지만, Dropouts은 network(large noise)를 수정)

References
1. [5 Techniques to Prevent Overfitting in Neural Networks](https://www.kdnuggets.com/2019/12/5-techniques-prevent-overfitting-neural-networks.html)
2. [Keras API reference / Layers API / Layer weight regularizers](https://keras.io/api/layers/regularizers/)
3. [Keras API reference / Layers API / Regularization layers / Dropout layer](https://keras.io/api/layers/regularization_layers/dropout/)

In [10]:
def classification_model(x_train, y_train, x_test_1, y_test_1, x_test_2, y_test_2, n_channel_1=32, n_channel_2=32, n_dense=64, n_train_epoch=10, regularizer_='L2', lambda_=0, dr=0):
    
        # Regularizer
        if regularizer_ == 'L1':
                regularizer=keras.regularizers.l1(l1=lambda_)
        elif regularizer_ == 'L2':
                regularizer=keras.regularizers.l2(l2=lambda_)
        else: # applies both L1 and L2 penalties
                regularizer=keras.regularizers.l1_l2(l1=lambda_, l2=lambda_)

        model=keras.models.Sequential()
        # Feature Extractor
        model.add(keras.layers.Conv2D(n_channel_1, (3,3), activation='relu', input_shape=(28,28,3)))
        model.add(keras.layers.MaxPool2D(2,2))
        model.add(keras.layers.Conv2D(n_channel_2, (3,3), activation='relu'))
        model.add(keras.layers.MaxPooling2D((2,2)))
        # Transformer-1D
        model.add(keras.layers.Flatten())
        # Classifier (apply method #4-5 to prevent overfitting)
        model.add(keras.layers.Dense(n_dense, activation='relu', kernel_regularizer=regularizer))
        model.add(keras.layers.Dropout(rate=dr))
        model.add(keras.layers.Dense(3, activation='softmax'))

        # model.summary()

        # Train model
        model.compile(optimizer='adam',
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])
        model.fit(x_train, y_train, epochs=n_train_epoch)

        if lambda_ == 0 and dr == 0:
                print("Non-Regularization")
        else:
                if lambda_ != 0:
                        print(f"{regularizer_} Regularization... lambda: {lambda_}")
                if dr != 0:
                        print(f"Dropout Regularization... dropout-rate: {dr}")
        print("Hyperparams... n_ch_1: {}, n_ch_2: {}, n_dense: {}, n_train_epoch: {}".format(n_channel_1, n_channel_2, n_dense, n_train_epoch))
        print('-------------------- test case 1 (New dataset) --------------------')
        test_loss, test_accuracy = model.evaluate(x_test_1, y_test_1, verbose=2)
        print("test_loss: {} ".format(test_loss))
        print("test_accuracy: {}".format(test_accuracy))
        # test case 2 (Split dataset)
        print('-------------------- test case 2 (Split dataset) --------------------')
        test_loss, test_accuracy = model.evaluate(x_test_2, y_test_2, verbose=2)
        print("test_loss: {} ".format(test_loss))
        print("test_accuracy: {}".format(test_accuracy))
        print('\n')
        

In [11]:
# L1, L2, L1_L2 Regularization
reg_lst = ['L1', 'L2', 'L1_L2']
lambda_lst = [0, 0.01, 0.001, 0.0001]
for regularizer in reg_lst:
    for lambda_ in lambda_lst:
        classification_model(x_train_norm, y_train, x_test_norm, y_test, x_test_split_norm, y_test_split, regularizer_=regularizer, lambda_=lambda_)

# Dropout Regularization
dr_arr = np.arange(0., 0.6, 0.1, dtype=np.float32)[::-1]
for dr in dr_arr:
    classification_model(x_train_norm, y_train, x_test_norm, y_test, x_test_split_norm, y_test_split, dr=dr)

Epoch 1/10
94/94 [==============================] - 0s 2ms/step - loss: 1.0620 - accuracy: 0.4283
Epoch 2/10
94/94 [==============================] - 0s 2ms/step - loss: 0.8415 - accuracy: 0.6187
Epoch 3/10
94/94 [==============================] - 0s 2ms/step - loss: 0.6000 - accuracy: 0.7517
Epoch 4/10
94/94 [==============================] - 0s 2ms/step - loss: 0.4350 - accuracy: 0.8333
Epoch 5/10
94/94 [==============================] - 0s 2ms/step - loss: 0.3749 - accuracy: 0.8543
Epoch 6/10
94/94 [==============================] - 0s 2ms/step - loss: 0.2742 - accuracy: 0.9027
Epoch 7/10
94/94 [==============================] - 0s 2ms/step - loss: 0.2116 - accuracy: 0.9393
Epoch 8/10
94/94 [==============================] - 0s 2ms/step - loss: 0.1589 - accuracy: 0.9547
Epoch 9/10
94/94 [==============================] - 0s 2ms/step - loss: 0.1219 - accuracy: 0.9673
Epoch 10/10
94/94 [==============================] - 0s 2ms/step - loss: 0.1000 - accuracy: 0.9730
Non-Regularization


## 3차 테스트 결과

################################################################################################
|Regularization|Training acc.(%)|(Case#1) Test acc.(%)|(Case#2) Test acc.(%)|
|:------------:|:--------------:|:-----------------:|:-----------------:|
|Non-regularization                             |98.21|59.50|96.67|
|L1-regularization ($\lambda=0.01$)             |32.20|33.33|33.33|
|L1-regularization ($\lambda=0.001$)            |90.87|44.67|88.67|
|L1-regularization ($\lambda=0.0001$)           |96.67|56.33|94.00|
|L2-regularization ($\lambda=0.01$)             |92.93|54.67|91.33|
|L2-regularization ($\lambda=0.001$)            |97.67|66.00|94.33|
|L2-regularization ($\lambda=0.0001$)           |97.97|61.67|96.00|
|L1_L2-regularization ($\lambda_{1,2}=0.01$)    |33.33|33.33|33.33|
|L1_L2-regularization ($\lambda_{1,2}=0.001$)   |88.40|42.67|86.33|
|L1_L2-regularization ($\lambda_{1,2}=0.0001$)  |96.63|62.33|95.33|
|Dropout (rate=0.5)                             |92.10|54.33|95.33|
|Dropout (rate=0.4)                             |95.23|55.33|96.67|
|Dropout (rate=0.3)                             |93.63|59.00|96.00|
|Dropout (rate=0.2)                             |97.00|59.33|97.67|
|Dropout (rate=0.1)                             |96.90|63.33|96.00|

- L1, L2, L1_L2 regularization에서는 $\lambda$가 0에 가까워질수록 학습/테스트 정확도 상승
- 다만, L1-regularization에서 $\lambda=0.01$일 때 학습이 거의 진행되지 않는 특성 보임
- Dropout의 경우에는 rate가 (0.1~) 0.2에서 높은 테스트 정확도를 보임
- L1/L1_L2 regularization보다 L2 regularization과 Dropout에서의 테스트 정확도가 더 높음
- 3차 테스트의 best test accuracy는 각 **66%**(test case#1, L2-regularization: 0.001), **98%**(test case#2, Dropout: 0.2)

################################################################################################


## 4차 테스트 진행

목표
- 3차 테스트의 best parameter (L2-regularization lambda=0.001 + dropout rate=0.2)를 활용한 hyperparameter tuning
- 모델의 complexcity를 줄이기 위해 feature map 채널 수와 layer의 유닛 수가 최대 64를 넘기지 않고,
- Early stopping을 위해 학습 iteration(n_train_epoch)는 최대 10으로 설정 (to prevent overfitting) 

In [12]:
parameter_lst = [16, 32, 64]
for n_ch_1 in parameter_lst: # 3x3x3 경우의 수
    for n_ch_2 in parameter_lst:
        for n_dense in parameter_lst:
            classification_model(x_train_norm, y_train, x_test_norm, y_test, x_test_split_norm, y_test_split, n_channel_1=n_ch_1, n_channel_2=n_ch_2, n_dense=n_dense, regularizer_='L2', lambda_=0.001, dr=0.2)


Epoch 1/10
94/94 [==============================] - 0s 3ms/step - loss: 1.1166 - accuracy: 0.3450
Epoch 2/10
94/94 [==============================] - 0s 2ms/step - loss: 1.0954 - accuracy: 0.3967
Epoch 3/10
94/94 [==============================] - 0s 3ms/step - loss: 1.0560 - accuracy: 0.4653
Epoch 4/10
94/94 [==============================] - 0s 3ms/step - loss: 0.9653 - accuracy: 0.5450
Epoch 5/10
94/94 [==============================] - 0s 3ms/step - loss: 0.8585 - accuracy: 0.6047
Epoch 6/10
94/94 [==============================] - 0s 3ms/step - loss: 0.7564 - accuracy: 0.6667
Epoch 7/10
94/94 [==============================] - 0s 3ms/step - loss: 0.6430 - accuracy: 0.7323
Epoch 8/10
94/94 [==============================] - 0s 3ms/step - loss: 0.5747 - accuracy: 0.7703
Epoch 9/10
94/94 [==============================] - 0s 2ms/step - loss: 0.5158 - accuracy: 0.7897
Epoch 10/10
94/94 [==============================] - 0s 2ms/step - loss: 0.4674 - accuracy: 0.8213
L2 Regularization..

## 4차 테스트 결과

################################################################################################   

    Epoch 1/10
    94/94 [==============================] - 0s 3ms/step - loss: 1.1535 - accuracy: 0.3633
    Epoch 2/10
    94/94 [==============================] - 0s 2ms/step - loss: 1.0620 - accuracy: 0.4750
    Epoch 3/10
    94/94 [==============================] - 0s 2ms/step - loss: 0.9160 - accuracy: 0.5893
    Epoch 4/10
    94/94 [==============================] - 0s 2ms/step - loss: 0.7673 - accuracy: 0.6773
    Epoch 5/10
    94/94 [==============================] - 0s 2ms/step - loss: 0.6171 - accuracy: 0.7540
    Epoch 6/10
    94/94 [==============================] - 0s 2ms/step - loss: 0.4800 - accuracy: 0.8343
    Epoch 7/10
    94/94 [==============================] - 0s 2ms/step - loss: 0.4021 - accuracy: 0.8740
    Epoch 8/10
    94/94 [==============================] - 0s 2ms/step - loss: 0.3442 - accuracy: 0.8897
    Epoch 9/10
    94/94 [==============================] - 0s 2ms/step - loss: 0.3005 - accuracy: 0.9140
    Epoch 10/10
    94/94 [==============================] - 0s 3ms/step - loss: 0.2417 - accuracy: 0.9437
    L2 Regularization... lambda: 0.001
    Dropout Regularization... dropout-rate: 0.2
    Hyperparams... n_ch_1: 32, n_ch_2: 16, n_dense: 64, n_train_epoch: 10
    -------------------- test case 1 (New dataset) --------------------
    10/10 - 0s - loss: 0.9594 - accuracy: 0.6967
    test_loss: 0.9593999981880188 
    test_accuracy: 0.6966666579246521
    -------------------- test case 2 (Split dataset) --------------------
    10/10 - 0s - loss: 0.2112 - accuracy: 0.9633
    test_loss: 0.21121320128440857 
    test_accuracy: 0.9633333086967468

    
- 4차 테스트의 best test accuracy는 각 **70%**(test case#1), **96%**(test case#2)
- 마지막으로 아쉬운 부분이 있다면, n_train_epoch를 조금 더 늘려서 위의 train accuracy를 조금 더 높일 수 있지 않을까?

################################################################################################

In [13]:
# n_train_epoch = 10 -> 12
classification_model(x_train_norm, y_train, x_test_norm, y_test, x_test_split_norm, y_test_split, n_channel_1=32, n_channel_2=16, n_dense=64, n_train_epoch=12, regularizer_='L2', lambda_=0.001, dr=0.2)

Epoch 1/12
94/94 [==============================] - 0s 2ms/step - loss: 1.1540 - accuracy: 0.3727
Epoch 2/12
94/94 [==============================] - 0s 3ms/step - loss: 1.0485 - accuracy: 0.4900
Epoch 3/12
94/94 [==============================] - 0s 3ms/step - loss: 0.8643 - accuracy: 0.6233
Epoch 4/12
94/94 [==============================] - 0s 2ms/step - loss: 0.6812 - accuracy: 0.7260
Epoch 5/12
94/94 [==============================] - 0s 3ms/step - loss: 0.5458 - accuracy: 0.7937
Epoch 6/12
94/94 [==============================] - 0s 2ms/step - loss: 0.4487 - accuracy: 0.8423
Epoch 7/12
94/94 [==============================] - 0s 2ms/step - loss: 0.3709 - accuracy: 0.8843
Epoch 8/12
94/94 [==============================] - 0s 2ms/step - loss: 0.3156 - accuracy: 0.9077
Epoch 9/12
94/94 [==============================] - 0s 2ms/step - loss: 0.2772 - accuracy: 0.9230
Epoch 10/12
94/94 [==============================] - 0s 2ms/step - loss: 0.2398 - accuracy: 0.9440
Epoch 11/12
94/94 [

- but, 결과는 늘어나지 않고 오히려 overfitting 되는 느낌
- 따라서 maximum n_train_epoch = 10이 가장 적절하다고 판단      


        L2 Regularization... lambda: 0.001
        Dropout Regularization... dropout-rate: 0.2

        Hyperparams... n_ch_1: 32, n_ch_2: 16, n_dense: 64, n_train_epoch: 10

        -------------------- test case 1 (New dataset) --------------------
        10/10 - 0s - loss: 0.9594 - accuracy: 0.6967
        test_loss: 0.9593999981880188 
        test_accuracy: 0.6966666579246521

        -------------------- test case 2 (Split dataset) --------------------
        10/10 - 0s - loss: 0.2112 - accuracy: 0.9633
        test_loss: 0.21121320128440857 
        test_accuracy: 0.9633333086967468

## 결론

- 학습된 모델에 2가지 test data를 돌려본 결과 최종 test accuracy는 각 **70%**, **96%** 였다.
- 분류 모델 학습 시 Overfitting을 피하기 위해 **L2 Regularization(0.001)** 및 **Dropout(0.2)** 을 적용하였으며,
- Hyperparameters는 **n_channel_1: 32, n_channel_2: 16(64), n_dense: 64, n_train_epoch: 10** 이다.   


- 분류기 모델을 학습시키면서 overfitting 방지를 위해 여러 가지를 시도했다. 가중치가 작은 값을 가지도록 규제(regularization)하여 가중치 값의 분포를 조금 더 균일하게 만들었으며, 이는 네트워크 복잡도를 완화시켰다고 생각한다. 또한 학습 시 일부 신경망을 생략(dropout)하여 특정 weight 조합에 의존하는 co-adaption 현상을 피해 overfitting 방지했다.

- 이번 노드를 진행하면서 초기에 어려웠던 부분은 학습을 돌리기 전에 데이터셋에 대한 이해가 충분하지 못했다는 점이다. label 당 1000장의 이미지가 있다고 하더라도 사실 단 몇 사람이 같은 배경 속에서 100장씩 연속 촬영하여 만든 이미지이므로 학습 시 특정 부분에 overfitting 될 소지가 높다. 학습 모델의 성능을 높이기 위해서 단순히 데이터의 양을 늘리는 것보다는 정제된 데이터를 다양하게 수집하는 것이 중요하다고 생각한다. 다음 데이터 수집 시에는 좀 더 다양한 배경의 데이터를 확보 후 학습을 진행할 것이며, 또한 이번 노드에서는 작게 resized (28x28) 이미지를 사용했지만 다음에는 크기를 키우는 등 이미지 크기에 따라 모델 성능 및 정확도를 비교하고 싶다. 무작정 학습 후 파라미터를 튜닝하는 것보다는 자신의 데이터셋에 대해서 충분히 이해를 한 후 oerfitting, regularization, dropout, hyperparameter 등을 고민해 본다면 보다 더 재밌게 이번 프로젝트를 진행할 수 있을 것이다.